In [1]:
import os
import glob
import pandas as pd
os.chdir("/Users/jisha/Desktop/Lambda_Course_Work/Coursework/Work/3. Labs/Workspace_Datasets/Datasets/Crime_Data/csv")

In [2]:
path = os.getcwd()

In [3]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [4]:
# combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
# export to csv
combined_csv.to_csv("combined_csv.csv", index=False)

In [5]:
import pandas as pd
import numpy as np

In [6]:
crime_filepath = os.path.join(path, "combined_csv.csv")
combined_csv = pd.read_csv(crime_filepath)
print(combined_csv.shape)
combined_csv.head()

(8105, 14)


,Unnamed: 0,City,State,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson
0,4,Acworth,GA,22921,16,0,2,1,13,437,24,399,14,0
1,5,Adairsville,GA,4953,7,0,2,0,5,83,15,59,9,0
2,6,Albany,GA,74989,790,12,32,165,581,3452,729,2489,234,17
3,7,Alma,GA,3438,10,0,0,2,8,111,36,69,6,0
4,8,Alpharetta,GA,67411,75,1,7,11,56,991,97,849,45,0


### Feature Engineering
Creating a crime percentage column 
- combines total crime/ population

In [7]:
# Population cannot be 0
combined_csv.loc[combined_csv['Population']==0.0]

,Unnamed: 0,City,State,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson
693,7,Baltimore City,MD,0,0,0,0,0,0,0,0,0,0,0
7055,77,Sunriver,OR,0,1,0,0,0,1,54,9,43,2,0


In [8]:
combined_csv = combined_csv[combined_csv.Population != 0]

In [9]:
# Crime is broken down into violent crime and property crime
# Arson is reported in conjunction with property or violent crime depending on the nature
columns = ['Violent crime', 'Property crime']

In [10]:
combined_csv[columns] = combined_csv[columns].astype(int)

In [11]:
def sum_frame_by_column(frame, new_col_name, list_of_cols_to_sum, divideby):
    frame[new_col_name] = ((frame[list_of_cols_to_sum].astype(int).sum(axis=1)/frame[divideby].astype(int))*100).round(2)
    return(frame)

In [12]:
sum_frame_by_column(combined_csv, 'Crime Percentage', columns, 'Population' )

,Unnamed: 0,City,State,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Crime Percentage
0,4,Acworth,GA,22921,16,0,2,1,13,437,24,399,14,0,1.98
1,5,Adairsville,GA,4953,7,0,2,0,5,83,15,59,9,0,1.82
2,6,Albany,GA,74989,790,12,32,165,581,3452,729,2489,234,17,5.66
3,7,Alma,GA,3438,10,0,0,2,8,111,36,69,6,0,3.52
4,8,Alpharetta,GA,67411,75,1,7,11,56,991,97,849,45,0,1.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8100,249,West Plains,MO,12284,52,0,10,2,40,709,126,496,87,0,6.20
8101,250,Wheaton,MO,693,0,0,0,0,0,0,0,0,0,0,0.00
8102,251,Willard,MO,5616,7,0,1,0,6,74,14,55,5,0,1.44
8103,252,Willow Springs,MO,2103,13,0,4,0,9,71,22,41,8,1,3.99


In [13]:
violent_crimes = ['Murder and nonnegligent manslaughter', 'Rape','Robbery', 
                  'Aggravated assault']

In [14]:
combined_csv['Violent crime']= combined_csv[violent_crimes].sum(axis=1)

In [15]:
property_crimes = ['Burglary', 'Larceny- theft', 'Motor vehicle theft']

In [16]:
combined_csv['Property crime']= combined_csv[property_crimes].sum(axis=1)

In [17]:
min(combined_csv['Crime Percentage']), max(combined_csv['Crime Percentage'])

(0.0, 2562.5)

In [18]:
# Just wanted to see the type of crime for such a high percentage
combined_csv.loc[combined_csv['Crime Percentage']==2562.5]

,Unnamed: 0,City,State,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Crime Percentage
4098,71,Lakeside,CO,8,0,0,0,0,0,205,1,202,2,0,2562.5


In [19]:
def crime_rating(row):
# Separating values into levels of concerns
  if 0 <= row['Crime Percentage'] <= 25:
    val = 'Low'
  elif 26 <= row['Crime Percentage'] <= 50:
    val = 'Medium'
  elif 51 <= row['Crime Percentage'] <= 75:
    val = 'High'
  else:
    val = 'Very High'

  return val

In [20]:
combined_csv['Crime Rating'] = combined_csv.apply(crime_rating, axis=1)

In [21]:
combined_csv.to_csv("fbi_crime_uscities.csv", index=False)

In [22]:
fbi_crime_filepath = os.path.join(path, "fbi_crime_uscities.csv")
fbi_crime_uscities = pd.read_csv(fbi_crime_filepath)
fbi_crime_uscities.head()

,Unnamed: 0,City,State,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Crime Percentage,Crime Rating
0,4,Acworth,GA,22921,16,0,2,1,13,437,24,399,14,0,1.98,Low
1,5,Adairsville,GA,4953,7,0,2,0,5,83,15,59,9,0,1.82,Low
2,6,Albany,GA,74989,790,12,32,165,581,3452,729,2489,234,17,5.66,Low
3,7,Alma,GA,3438,10,0,0,2,8,111,36,69,6,0,3.52,Low
4,8,Alpharetta,GA,67411,75,1,7,11,56,991,97,849,45,0,1.58,Low


In [23]:
fbi_crime_uscities.loc[fbi_crime_uscities['Crime Percentage']>=100]

,Unnamed: 0,City,State,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Crime Percentage,Crime Rating
4037,11,Black Hawk,CO,128,8,0,0,0,8,211,4,200,7,0,171.09,Very High
4097,71,Lakeside,CO,8,0,0,0,0,0,205,1,202,2,0,2562.50,Very High
5552,190,Industry,CA,201,72,0,1,41,30,1180,156,899,125,7,622.89,Very High
5793,431,Vernon,CA,112,27,0,1,12,14,485,133,272,80,5,457.14,Very High
6863,431,Teterboro,NJ,68,0,0,0,0,0,102,0,99,3,0,150.00,Very High


In [24]:
len(fbi_crime_uscities.loc[fbi_crime_uscities['Crime Percentage']>=100])

5